In [ ]:
# Global variables
gpu = False
device = "cuda" if gpu else "cpu"

# Loading the data with some normalisation
As the grid is fixed, we normalise X, Y, Z, T

T might be changed.

In [ ]:
# from inr_src import return_dataset
import inr_src as inr
path = "./data/test_data.npy"
ds, ds_test, nv, nv_y = inr.return_dataset(path, gpu=False)

print("##########\n X, Y, T")
print(ds.samples)
print("##########\n Z")
print(ds.targets)
print("##########\n Shape")
print(ds.targets.shape)

# Implicit Neural Representation

Currently, different models are available: RFF or SIREN. I could also implement WIRES (based on wavelet functions).
For these models, different archectures are available: 

The loss we are minimizing is the: $$\mathcal{L} = ||f(x, y, t)  - z ||_2 + \lambda_{xy}||\frac{\mathrm{d} f}{\mathrm{d} xy}||_2 + \lambda_{t}||\frac{\mathrm{d} f}{ \mathrm{d} t}||_2 $$ 

In [ ]:
path = "./data/test_data.npy"

opt = inr.AttrDict()
opt.path = path
opt.gpu = gpu
opt.fourier = True #False  #False
opt.siren = False # True
opt.name = "RFF"
model_hp = inr.AttrDict()

model_hp.fourier = opt.fourier
model_hp.siren = opt.siren
model_hp.verbose = True
model_hp.epochs = 50

model_hp.bs = 2**16
model_hp.scale = 5
model_hp.lr = 1e-5
model_hp.output_size = 1

if opt.siren:
    print("Using siren")
    model_hp.architecture = "siren"
    model_hp.siren_hidden_num = 5
    model_hp.siren_hidden_dim = 1024
    model_hp.siren_skip = True
else:
    model_hp.mapping_size = 512
    model_hp.architecture = "skip-5"  # "Vlarge"
    model_hp.activation = "tanh"

model_hp.lambda_t = 0.0
model_hp.lambda_xy = 0.0

model, model_hp = inr.train(opt, model_hp, gpu=opt.gpu)


In [ ]:
# Or if you prefer to load the model
import numpy as np
import torch
from inr_src.models import ReturnModel
from inr_src.project_parser import AttrDict
from inr_src.util_train import clean_hp

opt = AttrDict()
opt.name = "RFF"
## From saved
npz = np.load(f"meta/{opt.name}.npz")

weights = f"meta/{opt.name}.pth"
model_hp = AttrDict(npz)
model_hp = clean_hp(model_hp)

model = ReturnModel(
    int(model_hp.input_size),
    int(model_hp.output_size),
    arch=str(model_hp.architecture),
    args=model_hp,
)
model.load_state_dict(torch.load(weights))

# Some data visualisation
## Some real data

In [ ]:
import chart_studio.plotly as py
import ipywidgets as ipw
import plotly.express as px

# Visualisation the data set by tempoarl averaging of each month
pc = np.load("data/test_data.npy")
q33 = np.quantile(pc[:,3], 0.33)
q66 = np.quantile(pc[:,3], 0.66)
pc_t0 = pc[pc[:,3] < q33]
pc_t1 = pc[(pc[:,3] >= q33) & (pc[:,3] < q66)]
pc_t2 = pc[(pc[:,3] >= q66)]

for spc, title in [(pc_t0, "Jan."), (pc_t1, "Feb."), (pc_t2, "Mar.")]:

    idx = np.random.choice(np.arange(spc.shape[0]), replace=False, size=int(1e5))
    fig = px.scatter_3d(x=spc[idx,0], y=spc[idx,1], z=spc[idx,2],
                color=spc[idx,3])
    fig.update_layout(title=title, legend_title_text="Time")
    fig.update_traces(marker_size=2)
    fig.show()


## Trained model predicts Z from XYT of the trained points! (Should be good...)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

xytz_ds = inr.XYTZ(
        path,
        train_fold=False,
        train_fraction=0.0,
        seed=42,
        pred_type="pc",
        nv=model_hp.nv,
        nv_targets=model_hp.nv_target,
        gpu=gpu
    )

prediction = inr.predict_loop(xytz_ds, 2048, model, device=device)

mse_norm = mean_squared_error(xytz_ds.targets, prediction)
mae_norm = mean_absolute_error(xytz_ds.targets, prediction)

norm_target = xytz_ds.targets * model_hp.nv_target[0,1] + model_hp.nv_target[0,0]
norm_pred = prediction * model_hp.nv_target[0,1] + model_hp.nv_target[0,0]
mse_unnorm = mean_squared_error(norm_target, norm_pred)
mae_unnorm = mean_absolute_error(norm_target, norm_pred)

print(f"Normalised data -> MSE: {mse_norm:.5f} MAE: {mae_norm:.5f}")
print(f"True Z values ->   MSE: {mse_unnorm:.3f} MAE: {mae_unnorm:.3f}")

q33 = np.quantile(xytz_ds.samples[:,2], 0.33)
q66 = np.quantile(xytz_ds.samples[:,2], 0.66)

idx_0 = xytz_ds.samples[:,2] < q33
idx_1 = (xytz_ds.samples[:,2] >= q33) & (xytz_ds.samples[:,2] < q66)
idx_2 = (xytz_ds.samples[:,2] >= q66)

for idx, title in [(idx_0, "Jan."), (idx_1, "Feb."), (idx_2, "Mar.")]:
    samples = xytz_ds.samples[idx] * model_hp.nv[:,1] + model_hp.nv[:,0]
    pred = prediction[idx,0] * model_hp.nv_target[0,1] + model_hp.nv_target[0,0]

    idx = np.random.choice(np.arange(samples.shape[0]), replace=False, size=int(1e5))
    fig = px.scatter_3d(x=samples[idx,0], y=samples[idx,1], z=pred[idx],
                color=samples[idx,2])
    fig.update_layout(title=title, legend_title_text="Time")
    fig.update_traces(marker_size=2)
    fig.show()


# Predicting along the grid with "t = middle of the month"

In [ ]:
# Compute linear grid for plotting
#Capture a month
q16 = np.quantile(pc[:,3], 0.1666)
q50 = np.quantile(pc[:,3], 0.50)
q83 = np.quantile(pc[:,3], 0.8333)

dataset16 = inr.XYTZ_grid(pc, q16, nv=model_hp.nv, step_grid=0.01, gpu=gpu)
dataset50 = inr.XYTZ_grid(pc, q50, nv=model_hp.nv, step_grid=0.01, gpu=gpu)
dataset83 = inr.XYTZ_grid(pc, q83, nv=model_hp.nv, step_grid=0.01, gpu=gpu)
nv = np.array(model_hp.nv)
nv_target = np.array(model_hp.nv_target)
XYT16 = dataset16.samples * nv[:,1] + nv[:,0]
device = "cuda" if gpu else "cpu"
# Predict along the grid
preds16 = inr.predict_loop(dataset16, 2048, model, device=device)
preds50 = inr.predict_loop(dataset50, 2048, model, device=device)
preds83 = inr.predict_loop(dataset83, 2048, model, device=device)

for spred, title in [(preds16, "Jan."), (preds50, "Feb."), (preds83, "Mar.")]:
    spred_z = spred * nv_target[0,1] + nv_target[0,0]
    fig = px.scatter_3d(x=XYT16[:,0], y=XYT16[:,1], z=spred_z[:,0],
                color=spred_z[:,0])
    fig.update_layout(title=title, legend_title_text="Time")
    fig.update_traces(marker_size=2)
    fig.show()